In [40]:
# libraries to visualize
import matplotlib.pyplot as plt

# Generates a confusion matrix
from sklearn.linear_model import LinearRegression, Ridge, Lasso

## some stuff for making pretty plots
from matplotlib import rcParams
from cycler import cycler

## Set plotting style and print options
sns.set_theme()
sns.set_style("whitegrid")
sns.set_context("paper") #Possible are paper, notebook, talk and poster
# Set figure format
wide_format, square_format = False, True
if wide_format:
    d = {'lines.linewidth': 2, 'axes.titlesize': 18, 'axes.labelsize': 18, 'xtick.labelsize': 12, 'ytick.labelsize': 12,\
     'legend.fontsize': 15, 'font.family': 'serif', 'figure.figsize': (9,6)}
if square_format:
    d = {'lines.linewidth': 2, 'axes.titlesize': 16, 'axes.labelsize': 14, 'xtick.labelsize': 10, 'ytick.labelsize': 10,\
     'legend.fontsize': 12, 'font.family': 'serif', 'figure.figsize': (6,6)}
    
d_colors = {'axes.prop_cycle': cycler(color = ['teal', 'navy', 'coral', 'plum', 'purple', 'olivedrab',\
         'black', 'red', 'cyan', 'yellow', 'khaki','lightblue'])}
rcParams.update(d)
rcParams.update(d_colors)

# SalesInsightsPreprocessing
from SamplingScript import *

In [43]:
df

,Account Description ID,Type,Account Description,Size,Size Numeric,Strength (mg),WHO ATC 5 Code,Volume,Value,Year Month (after 2000),Year Month (after 2000) in Datetime,Region_Hovedstaden,Region_Midtjylland,Region_Nordjylland,Region_Sjælland,Region_Syddanmark,Product_Cimzia,Product_Inflectra,Product_Remicade,Product_Remsima,Product_Stelara,Product_Zessly
0,0,0,4070 Sjællands Universitetshospital. Roskilde,1 htgl. a 100 mg,100.0,100.0,0,238.0,1048818.40,1001.0,2010-01,0,0,0,1,0,0,0,1,0,0,0
1,0,0,4070 Sjællands Universitetshospital. Roskilde,1 htgl. a 100 mg,100.0,100.0,0,206.0,907800.80,1002.0,2010-02,0,0,0,1,0,0,0,1,0,0,0
2,0,0,4070 Sjællands Universitetshospital. Roskilde,2 stk.,2.0,200.0,1,8.0,64668.40,1003.0,2010-03,0,0,0,1,0,1,0,0,0,0,0
3,0,0,4070 Sjællands Universitetshospital. Roskilde,1 htgl. a 100 mg,100.0,100.0,0,350.0,1542380.00,1003.0,2010-03,0,0,0,1,0,0,0,1,0,0,0
4,0,0,4070 Sjællands Universitetshospital. Roskilde,2 stk.,2.0,200.0,1,18.0,145503.90,1004.0,2010-04,0,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6871,258,1,4398 Viborg Sygehus. apoteket,1 htgl.,1.0,100.0,0,472.0,2028009.36,1310.0,2013-10,0,1,0,0,0,0,0,1,0,0,0
6872,258,1,4398 Viborg Sygehus. apoteket,2 stk.,2.0,200.0,1,45.0,354665.70,1311.0,2013-11,0,1,0,0,0,1,0,0,0,0,0
6873,258,1,4398 Viborg Sygehus. apoteket,1 htgl.,1.0,100.0,0,410.0,1761618.30,1311.0,2013-11,0,1,0,0,0,0,0,1,0,0,0
6874,258,1,4398 Viborg Sygehus. apoteket,2 stk.,2.0,200.0,1,52.0,409835.92,1312.0,2013-12,0,1,0,0,0,1,0,0,0,0,0


In [44]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc 


clf = DecisionTreeClassifier(criterion="entropy", max_depth=10)


In [ ]:
for i in range(1, 100):
    for j in range(1, 15):
        DecisionTreeClassifier(criterion="entropy", max_depth=j)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
        clf.fit(X_train, y_train)
        if evaluate_model(clf, X_test, y_test)['Accuracy'] > 0.5:
            print(i)
            print(j)
            print(evaluate_model(clf, X_test, y_test))
            break